In [1]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import wandb
from ignite.engine import (Engine, Events, create_supervised_evaluator,
                           create_supervised_trainer)
from ignite.metrics import Accuracy, Loss
from scipy.io.arff import loadarff
from sklearn.model_selection import train_test_split
from torch import nn
from torch.functional import F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler

sys.path.append('../')
from src.datasets import FordDataset
from src.models import LSTMClassification

In [59]:
train_path = "../data/FordA/FordA_TRAIN.arff"
test_path = "../data/FordA/FordA_TEST.arff"

train_dataset = FordDataset(train_path)
test_dataset = FordDataset(test_path)

idx = np.arange(len(train_dataset))
idx_train, idx_val = train_test_split(idx, train_size=0.8, stratify=train_dataset.labels)

train_sampler = SubsetRandomSampler(idx_train)
val_sampler = SubsetRandomSampler(idx_val)

train_dataloader = DataLoader(train_dataset, batch_size=128, sampler=train_sampler)
val_dataloader = DataLoader(train_dataset, batch_size=500, sampler=val_sampler)
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [69]:
# Initialize your model
wandb.init(entity='ts-robustness', project='ml-course')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = LSTMClassification(1, 100, 1).to(device)

# Initialize your optimizer and criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch[0].to(device), batch[1].to(device)
    y_pred = model(x)
    loss = criterion(y_pred, y.unsqueeze(1))
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_step)

def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)
        return y_pred, y

train_evaluator = Engine(validation_step)
val_evaluator = Engine(validation_step)

# Attach metrics to the evaluators
metrics = {
    'accuracy_0.3': Accuracy(output_transform=lambda x: (x[0] > 0.3, x[1])),
    'accuracy_0.5': Accuracy(output_transform=lambda x: (x[0] > 0.5, x[1])),
    'accuracy_0.7': Accuracy(output_transform=lambda x: (x[0] > 0.7, x[1])),
    'loss': Loss(criterion, output_transform=lambda x: (x[0], x[1].unsqueeze(1)))
}

for name, metric in metrics.items():
    metric.attach(train_evaluator, name)

for name, metric in metrics.items():
    metric.attach(val_evaluator, name)

# # Logging
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(trainer):
    batch_loss = trainer.state.output
    print("Training Results - Avg loss: {:.4f}".format(batch_loss))
    wandb.log({"train_loss": batch_loss})
    
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    train_evaluator.run(train_dataloader)
    metrics = train_evaluator.state.metrics
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.4f}"
          .format(trainer.state.epoch, metrics['accuracy_0.5'], metrics['loss']))
    wandb.log({"train_accuracy_0.3": metrics['accuracy_0.3'],
               "train_accuracy_0.5": metrics['accuracy_0.5'],
               "train_accuracy_0.7": metrics['accuracy_0.7'],
               "train_loss": metrics['loss']})

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    val_evaluator.run(val_dataloader)
    metrics = val_evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.4f}"
          .format(trainer.state.epoch, metrics['accuracy_0.5'], metrics['loss']))
    wandb.log({"val_accuracy_0.3": metrics['accuracy_0.3'],
               "val_accuracy_0.5": metrics['accuracy_0.5'],
               "val_accuracy_0.7": metrics['accuracy_0.7'],
               "val_loss": metrics['loss']})

# Run the training loop
trainer.run(train_dataloader, max_epochs=30)
wandb.finish()

Training Results - Avg loss: 0.6942
Training Results - Avg loss: 3.8381
Training Results - Avg loss: 34.9338
Training Results - Avg loss: 17.6468
Training Results - Avg loss: 1.1535
Training Results - Avg loss: 1.0130
Training Results - Avg loss: 0.8180
Training Results - Avg loss: 0.7936
Training Results - Avg loss: 0.6779
Training Results - Avg loss: 0.6843
Training Results - Avg loss: 0.6984
Training Results - Avg loss: 0.7231
Training Results - Avg loss: 0.6982
Training Results - Avg loss: 0.7057
Training Results - Avg loss: 0.6996
Training Results - Avg loss: 0.6714
Training Results - Avg loss: 0.6717
Training Results - Avg loss: 0.6663
Training Results - Avg loss: 0.7072
Training Results - Avg loss: 0.7070
Training Results - Avg loss: 0.6786
Training Results - Avg loss: 0.6766
Training Results - Avg loss: 0.6810
Training Results - Avg loss: 0.6962
Training Results - Avg loss: 0.6814
Training Results - Avg loss: 0.6815
Training Results - Epoch: 1  Avg accuracy: 0.54 Avg loss: 0.68

train_accuracy_0.3,▁▂▂▂▃▃▂▃▄▄▄▄▅▅▅▅▅▅▆▆▆▇▆▇██████
train_accuracy_0.5,▁▂▃▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇█████████
train_accuracy_0.7,▁▂▃▂▃▃▃▄▅▄▄▅▆▆▆▆▆▇▇▇▇▇▇█▇█████
train_loss,█▇▆▆▇▇▆▆▆▅▅▅▄▆▅▄▄▄▄▄▄▂▄▅▃▃▃▂▃▂▃▃▂▃▃▂▁▂▁▁
val_accuracy_0.3,▁▂▄▂▃▄▄▃▅▅▅▅▆▆▅▅▅▅▆▇▇▇▇▇██▇▇██
val_accuracy_0.5,▁▃▅▅▆▅▇▆▇▆▇▆▇▇▇▇█▇▆███▇▆▇▇▇▇██
val_accuracy_0.7,▁▃▃▃▄▃▃▃▆▄▄▇▇▇▆▇▆▇▇▇█▇█▇▆▇█▇██
val_loss,▂▂▂▁▁▁▁▂▁▁▁▂▁▂▁▂▁▂▃▃▃▄▃▅▄▅▅▅█▆
train_accuracy_0.3,0.96944
train_accuracy_0.5,0.98951
train_accuracy_0.7,0.98765
